# 기본 에이전트: 도구를 사용하는 AI

이 노트북에서는 **LangGraph**를 사용하여 **도구(Tool)를 사용하는 에이전트**를 만듭니다.

## 에이전트(Agent)란?

```
┌────────────────────────────────────────────────────────────────────┐
│                    에이전트의 개념                                  │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 챗봇:                                                       │
│   사용자 질문 → LLM → 답변                                        │
│   (LLM이 알고 있는 정보로만 답변)                                  │
│                                                                    │
│   에이전트:                                                        │
│   사용자 질문 → LLM → 도구 호출 → 결과 확인 → 답변                │
│   (외부 도구를 사용해서 실시간 정보 획득 가능!)                    │
│                                                                    │
│   에이전트 = LLM + 도구 사용 능력                                  │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## 이번에 만들 시스템 (ReAct 패턴)

```
┌────────────────────────────────────────────────────────────────────┐
│                    ReAct 에이전트 아키텍처                          │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│                    ┌─────────┐                                     │
│                    │  START  │                                     │
│                    └────┬────┘                                     │
│                         │                                          │
│                         ▼                                          │
│                ┌────────────────┐                                  │
│                │     model      │ ◄─────────────┐                  │
│                │  (LLM + Tools) │               │                  │
│                └────────┬───────┘               │                  │
│                         │                       │                  │
│              tools_condition?                   │                  │
│                    ╱    ╲                       │                  │
│            도구 필요   도구 불필요              │                  │
│                 ╱          ╲                    │                  │
│                ▼            ▼                   │                  │
│         ┌──────────┐   ┌─────────┐             │                  │
│         │  tools   │   │   END   │             │                  │
│         │ (도구실행)│   └─────────┘             │                  │
│         └─────┬────┘                           │                  │
│               │                                │                  │
│               └────────────────────────────────┘                  │
│                      (결과를 다시 model로)                         │
│                                                                    │
│   ReAct = Reason + Act (추론하고 행동하기)                         │
│   LLM이 스스로 도구가 필요한지 판단하고, 필요하면 사용함           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph duckduckgo-search langchain-community

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 도구(Tool) 정의

에이전트가 사용할 도구들을 정의합니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    도구(Tool)의 구성요소                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   @tool 데코레이터로 함수를 도구로 변환                            │
│                                                                    │
│   필수 요소:                                                       │
│   • 함수 이름: LLM이 도구를 식별하는 데 사용                       │
│   • docstring: LLM이 도구의 용도를 이해하는 데 사용 (매우 중요!)   │
│   • 파라미터: 도구에 전달할 입력값                                 │
│   • 반환값: 도구 실행 결과                                         │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
import ast
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun

# 도구 1: 계산기
@tool
def calculator(query: str) -> str:
    '''계산기. 수식만 입력받습니다.'''
    return ast.literal_eval(query)

# 도구 2: 웹 검색 (DuckDuckGo)
search = DuckDuckGoSearchRun()

# 도구 리스트
tools = [search, calculator]

print("✅ 도구 정의 완료")
print(f"   - search: 웹에서 정보 검색")
print(f"   - calculator: 수학 계산")

In [ ]:
# 도구 테스트
print("=== 도구 테스트 ===")
print(f"계산기: 2 + 3 * 4 = {calculator.invoke('2 + 3 * 4')}")
print(f"\n검색: {search.invoke('Python programming')[:200]}...")

# 3. LLM에 도구 연결 (bind_tools)

LLM이 도구를 사용할 수 있도록 연결합니다.

In [ ]:
from langchain_ollama import ChatOllama

# LLM 모델에 도구 연결
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)

print("✅ LLM에 도구 연결 완료")
print("   bind_tools()로 LLM이 도구를 호출할 수 있게 됨")

# 4. State 정의

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    에이전트의 상태
    
    messages: 대화 기록 (사용자 질문, LLM 응답, 도구 결과 등)
    """
    messages: Annotated[list, add_messages]

print("✅ State 정의 완료")

# 5. 노드 정의

```
┌────────────────────────────────────────────────────────────────────┐
│                    두 가지 핵심 노드                               │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ model_node (직접 구현)                                       │
│      • LLM을 호출하여 응답 생성                                    │
│      • 도구가 필요하면 tool_calls 포함된 응답 반환                 │
│                                                                    │
│   2️⃣ ToolNode (LangGraph 제공)                                    │
│      • tool_calls를 받아서 실제로 도구 실행                        │
│      • 결과를 ToolMessage로 반환                                   │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langgraph.prebuilt import ToolNode

def model_node(state: State) -> State:
    """
    LLM 호출 노드
    
    1. 현재 대화 기록을 LLM에 전달
    2. LLM이 응답 생성 (도구 호출 포함 가능)
    3. 응답을 State에 추가
    """
    res = model.invoke(state['messages'])
    return {'messages': res}

# LangGraph가 제공하는 ToolNode 사용
tool_node = ToolNode(tools)

print("✅ 노드 정의 완료")
print("   - model_node: LLM 호출")
print("   - tool_node: 도구 실행 (ToolNode 사용)")

# 6. 그래프 구성

```
┌────────────────────────────────────────────────────────────────────┐
│                    tools_condition 이해하기                        │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   tools_condition은 LangGraph가 제공하는 조건 함수                 │
│                                                                    │
│   LLM 응답에 tool_calls가 있으면:                                  │
│   → "tools" 노드로 이동 (도구 실행)                                │
│                                                                    │
│   tool_calls가 없으면:                                             │
│   → "__end__" 로 이동 (종료)                                       │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition

# 그래프 빌더 생성
builder = StateGraph(State)

# 노드 추가
builder.add_node('model', model_node)
builder.add_node('tools', tool_node)

# 엣지 추가
builder.add_edge(START, 'model')           # 시작 → 모델
builder.add_conditional_edges('model', tools_condition)  # 조건부 분기
builder.add_edge('tools', 'model')         # 도구 → 모델 (루프)

# 그래프 컴파일
graph = builder.compile()

print("✅ 에이전트 그래프 컴파일 완료")

In [ ]:
# 그래프 구조 시각화
print("=== 에이전트 그래프 구조 ===")
print(graph.get_graph().draw_mermaid())

# 7. 에이전트 실행

In [ ]:
from langchain_core.messages import HumanMessage

# 테스트: 검색이 필요한 질문
input_data = {
    'messages': [
        HumanMessage(
            '미국의 제30대 대통령이 사망했을 때 몇 살이었나요?'
        )
    ]
}

print("=== 에이전트 실행 (스트림) ===")
print(f"질문: {input_data['messages'][0].content}\n")

for chunk in graph.stream(input_data):
    for node_name, node_output in chunk.items():
        print(f"--- {node_name} ---")
        if 'messages' in node_output:
            for msg in node_output['messages'] if isinstance(node_output['messages'], list) else [node_output['messages']]:
                print(f"  유형: {type(msg).__name__}")
                if hasattr(msg, 'content') and msg.content:
                    print(f"  내용: {str(msg.content)[:200]}..." if len(str(msg.content)) > 200 else f"  내용: {msg.content}")
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    print(f"  도구 호출: {msg.tool_calls}")
        print()

In [ ]:
# 최종 결과만 확인
result = graph.invoke(input_data)

print("=== 최종 응답 ===")
print(result['messages'][-1].content)

# 8. 다양한 질문으로 테스트

In [ ]:
test_questions = [
    "123 * 456 + 789는 얼마인가요?",        # 계산기 필요
    "오늘 서울 날씨는 어떤가요?",           # 검색 필요
    "안녕하세요, 반갑습니다!",              # 도구 불필요
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"질문: {question}")
    print("="*60)
    
    result = graph.invoke({'messages': [HumanMessage(question)]})
    
    # 어떤 도구가 사용되었는지 확인
    tools_used = []
    for msg in result['messages']:
        if hasattr(msg, 'tool_calls') and msg.tool_calls:
            tools_used.extend([tc['name'] for tc in msg.tool_calls])
    
    print(f"\n사용된 도구: {tools_used if tools_used else '없음'}")
    print(f"\n응답: {result['messages'][-1].content[:300]}..." if len(result['messages'][-1].content) > 300 else f"\n응답: {result['messages'][-1].content}")

---

## 정리: 기본 에이전트 (ReAct 패턴)

### 아키텍처

```
┌─────────────────────────────────────────────────────────────────────┐
│                    ReAct 에이전트 동작 흐름                         │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   1. 사용자 질문 입력                                               │
│   2. LLM이 질문 분석                                                │
│   3. 도구가 필요하면 tool_calls 생성                                │
│   4. ToolNode가 도구 실행                                           │
│   5. 결과를 다시 LLM에 전달                                         │
│   6. LLM이 최종 답변 생성 (또는 추가 도구 호출)                     │
│   7. 도구 호출이 없으면 종료                                        │
│                                                                     │
│   핵심: LLM이 스스로 도구 사용 여부를 결정!                         │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 핵심 코드

```python
# 1. 도구 정의
@tool
def my_tool(query: str) -> str:
    '''도구 설명 (LLM이 이해할 수 있게)'''
    return result

# 2. LLM에 도구 연결
model = ChatOllama(model='llama3.2').bind_tools(tools)

# 3. 조건부 엣지로 루프 구성
builder.add_conditional_edges('model', tools_condition)
builder.add_edge('tools', 'model')  # 결과를 다시 모델로
```

### 핵심 컴포넌트

| 컴포넌트 | 역할 | 제공 |
|----------|------|------|
| **@tool** | 함수를 도구로 변환 | LangChain |
| **bind_tools()** | LLM에 도구 연결 | LangChain |
| **ToolNode** | 도구 실행 노드 | LangGraph |
| **tools_condition** | 도구 호출 여부 판단 | LangGraph |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.1).bind_tools(tools)

# 변경
model = ChatOllama(model='llama3.2', temperature=0.1).bind_tools(tools)
```

## 다음 단계

**첫 번째 도구를 강제로 호출**하는 방법을 배웁니다. (03-04번 노트북)